In [18]:
import os
import numpy as np
import pandas as pd
import itertools
from datetime import datetime
from typing import Iterable
from typing import List, Optional
import matplotlib.pyplot as plt  

In [2]:
traits = pd.read_csv('./maize/maize_mean_pheno.csv')
traits

,Variety_ID,grain.yield,grain.number,seed.size,anthesis,silking,plant.height,tassel.height,ear.height
0,11430,6.714175,2524.895871,300.294106,65.329162,68.088152,214.287692,244.776356,108.314869
1,A3,6.283445,2687.746107,267.936893,66.266127,72.116642,221.942849,246.568521,112.339860
2,A310,5.648845,2098.978720,304.175569,66.237934,69.250734,221.825845,240.521063,124.086819
3,A347,5.493820,2119.242474,291.957786,67.707086,70.470427,209.095458,243.122323,114.835957
4,A374,7.505574,3357.301585,258.256427,67.818736,70.948375,218.698006,245.216113,114.789468
...,...,...,...,...,...,...,...,...,...
241,W604S,6.493161,2465.328005,302.815798,68.165436,71.256180,218.385125,249.157640,116.467945
242,W64A,6.386390,2560.871368,294.354638,65.585955,69.247288,208.377146,237.077976,107.666317
243,W9,5.061490,2346.467710,246.163726,63.539328,68.091173,210.127044,232.934226,106.582606
244,W95115,5.621730,2142.086821,292.216995,65.304476,69.832943,208.493574,229.963484,105.211562


In [3]:
markers = pd.read_csv('./maize/maize_markers.csv')
markers

,individual,SYN83,PZE.101000060,PZE.101000088,PZE.101000083,PZE.101000108,PZE.101000111,PZE.101000161,PZE.101000169,PZE.101000209,...,PZE.110111381,PZE.110111385,PZE.110111388,PZE.110111408,PZE.110111412,PZE.110111417,PZE.110111438,PZE.110111440,PZE.110111453,PZE.110111485
0,11430,0,2,2,0,0,2,2,0,0,...,2,2,2,2,2,2,2,2,2,2
1,A3,0,2,2,0,0,2,2,0,0,...,0,0,0,0,0,0,0,0,0,0
2,A310,2,2,2,2,2,2,2,0,0,...,2,2,2,2,2,2,2,2,2,2
3,A347,2,0,2,2,2,2,1,0,0,...,2,2,2,2,2,2,2,2,2,2
4,A374,2,0,2,2,2,2,0,2,2,...,2,2,2,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,W604S,0,2,2,0,2,0,2,0,2,...,2,2,2,2,2,2,2,2,2,2
242,W64A,0,2,2,0,0,2,2,0,0,...,0,0,0,0,0,0,0,0,0,0
243,W9,2,2,2,0,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
244,W95115,0,2,2,0,2,2,2,0,2,...,1,2,1,1,1,1,1,1,1,1


In [4]:
def snp_heterozygosity(df, count_nans_as_invalid=True):
    """
    Computes observed and expected heterozygosity per column and the inbreeding
    coefficient F.

    Accepts SNP genotypes encoded as:
      0 -> homozygote REF/REF
      1 -> heterozygote REF/ALT
      2 -> homozygote ALT/ALT

    PARAMETERS:
      df: DataFrame with the data
      count_nans_as_invalid: if True, NaNs are excluded from the denominator

    RETURNS:
      DataFrame with Ho and He per locus and average.

    It also creates:
      - global variable F
      - global variable heterozygosity_table
    """
    
    # Automatically use the first column's name as the column to ignore
    ignore_cols = (df.columns[0],)

    cols = [c for c in df.columns if c not in ignore_cols]

    observed = {}
    expected = {}

    for col in cols:
        series_raw = df[col]

        # Denominator based on parameter count_nans_as_invalid
        if count_nans_as_invalid:
            # Exclude NaNs from the denominator
            series_valid = series_raw.dropna()
            total = len(series_valid)
        else:
            # Include NaNs in the denominator
            series_valid = series_raw.dropna()
            total = len(series_raw)

        if total == 0:
            observed[col] = np.nan
            expected[col] = np.nan
            continue

        # Convert entries to 0/1/2 if valid
        def to_int_if_valid(x):
            try:
                if pd.isna(x):
                    return np.nan
                xi = int(x)
                if xi in (0, 1, 2):
                    return xi
                return np.nan
            except Exception:
                return np.nan

        genotypes_valid = series_valid.map(to_int_if_valid).dropna().astype(int)

        # Observed heterozygosity (Ho): proportion of heterozygotes (1)
        n_het = (genotypes_valid == 1).sum()
        Ho = n_het / total if total > 0 else np.nan
        observed[col] = Ho

        # Expected heterozygosity (He): 2*p*q
        n0 = (genotypes_valid == 0).sum()  # homozygote REF/REF
        n1 = (genotypes_valid == 1).sum()  # heterozygote REF/ALT
        n2 = (genotypes_valid == 2).sum()  # homozygote ALT/ALT
        n_valid = n0 + n1 + n2  # number of valid genotypes

        if n_valid == 0:
            He = np.nan
        else:
            # Allele counts: REF allele = 2*n0 + n1 ; ALT allele = 2*n2 + n1
            ref_count = 2 * n0 + n1
            alt_count = 2 * n2 + n1
            denom = 2 * n_valid  # Total number of alleles
            p = ref_count / denom
            q = alt_count / denom
            He = 1 - (p**2 + q**2)  # equivalent to 2*p*q

        expected[col] = He

    # Build final result table
    result = pd.DataFrame(
        [observed, expected],
        index=["observed_heterozygosity", "expected_heterozygosity"]
    )
    result["average"] = result.mean(axis=1)

    # Compute global averages and F
    Ho_avg = result.loc["observed_heterozygosity", cols].mean()
    He_avg = result.loc["expected_heterozygosity", cols].mean()
    F_value = 1 - (Ho_avg / He_avg) if (He_avg is not None and not np.isnan(He_avg) and He_avg != 0) else np.nan

    # Create global variable F
    globals()['F'] = F_value

    # Create global variable heterozygosity_table
    globals()['heterozygosity_table'] = result

    print("Inbreeding coefficient F:", F_value)
    print("")

    return result

In [5]:
snp_heterozygosity(markers)

Inbreeding coefficient F: 0.9740991551394081



,SYN83,PZE.101000060,PZE.101000088,PZE.101000083,PZE.101000108,PZE.101000111,PZE.101000161,PZE.101000169,PZE.101000209,PZE.101000256,...,PZE.110111385,PZE.110111388,PZE.110111408,PZE.110111412,PZE.110111417,PZE.110111438,PZE.110111440,PZE.110111453,PZE.110111485,average
observed_heterozygosity,0.077236,0.024390,0.004065,0.004065,0.008130,0.000000,0.028455,0.008130,0.008130,0.016260,...,0.008130,0.008130,0.008130,0.008130,0.008130,0.008130,0.004065,0.008130,0.008130,0.009670
expected_heterozygosity,0.481749,0.400026,0.269573,0.487433,0.176086,0.288486,0.382998,0.484004,0.298929,0.417377,...,0.314099,0.319023,0.298929,0.298929,0.298929,0.298929,0.296343,0.304052,0.298929,0.373346


In [30]:
def snp_pairwise_relatedness(df=None, decimals: int = 4) -> pd.DataFrame:
    """
    Compute pairwise relatedness as Pearson correlation between rows (individuals).

    Notes:
      - First column is treated as the sample ID.
      - The remaining columns are SNPs coded as 0/1/2.
      - Uses pairwise-complete Pearson correlation.
      - Produces global variables r and relatedness_table.
    """

    print("Step 1/6: Checking input dataframe...")
    global df_gen
    if df is None:
        df = df_gen

    if df is None:
        raise ValueError("No dataframe provided and global df_gen is not defined.")

    nrows = len(df)
    if nrows < 2:
        print("Not enough rows to compute pairwise relatedness.")
        r = pd.DataFrame(columns=["pair", "relatedness"])
        globals()['r'] = r
        globals()['relatedness_table'] = r
        return r

    print("Step 2/6: Extracting individual IDs (first column)...")
    id_col = df.columns[0]
    ids = df[id_col].astype(str).tolist()

    print("Step 3/6: Converting SNP data to numeric format...")
    # Convert remaining columns to numeric (0/1/2), invalid -> NaN
    data = df.drop(columns=[id_col]).apply(pd.to_numeric, errors='coerce')

    print("Step 4/6: Computing correlation matrix (this may take a while)...")
    # Transpose so columns = individuals
    data_T = data.T
    data_T.columns = ids

    corr_matrix = data_T.corr(method='pearson')

    print("Step 5/6: Cleaning correlation matrix (replacing NaN with 0.0)...")
    corr_matrix_filled = corr_matrix.fillna(0.0)

    print("Step 6/6: Building pairwise relatedness output...")
    results = []
    cols = corr_matrix_filled.columns.tolist()

    for a, b in itertools.combinations(cols, 2):
        val = corr_matrix_filled.at[a, b]
        results.append({
            "pair": f"{a}_{b}",
            "relatedness": round(float(val), decimals)
        })

    r = pd.DataFrame(results)
    globals()['r'] = r
    globals()['relatedness_table'] = r

    print("Done! Pairwise relatedness successfully computed.")
    return r


In [15]:
snp_pairwise_relatedness(markers)

Step 1/6: Checking input dataframe...
Step 2/6: Extracting individual IDs (first column)...
Step 3/6: Converting SNP data to numeric format...
Step 4/6: Computing correlation matrix (this may take a while)...
Step 5/6: Cleaning correlation matrix (replacing NaN with 0.0)...
Step 6/6: Building pairwise relatedness output...
Done! Pairwise relatedness successfully computed.


,pair,relatedness
0,11430_A3,0.1015
1,11430_A310,0.0668
2,11430_A347,0.0751
3,11430_A374,0.0574
4,11430_A375,0.0757
...,...,...
30130,W64A_W95115,0.1018
30131,W64A_Wf9,0.5474
30132,W9_W95115,0.1008
30133,W9_Wf9,0.0731


In [21]:
def phenotypic_similarity(
    df: Optional[pd.DataFrame],
    trait_cols: Optional[List[str]] = None,
    decimals: int = 4,
) -> pd.DataFrame:
    """
    Compute bounded phenotypic similarity in [0,1] for each trait and each pair of rows,
    using R_k = max_k - min_k (range) for normalization.

    similarity_k(i,j) = (R_k - abs(x_i - x_j)) / R_k

    Behavior:
    - If df is None, the function will try to use a global DataFrame named `mor`.
      If `mor` is not found, a ValueError is raised.
    - If trait_cols is None, numeric columns are auto-selected excluding the FIRST
      column (treated as the sample/individual ID).
    - If R_k == 0 (all values identical) similarity is set to 1 for that trait.
    - If either x_i or x_j is NaN → similarity is NaN for that trait.
    - Output: DataFrame with "pair" column (using values from the FIRST column if present,
      otherwise 1-based indices "i_j") and one column per trait.
    - Final DataFrame is assigned to the global variable `z` and also returned.

    Parameters
    ----------
    df : pd.DataFrame or None
        Input dataframe containing phenotypic trait columns. If None, uses global `mor`.
    trait_cols : list[str] | None
        List of trait column names to use. If None, auto-select numeric columns
        (excluding the first column).
    decimals : int
        Number of decimals to round similarity values.

    Returns
    -------
    pd.DataFrame
        DataFrame named `z` with columns: 'pair' and one column per selected trait.
    """
    # fallback to global 'mor' if df not provided
    if df is None:
        if 'mor' not in globals():
            raise ValueError(
                "df not provided and no global DataFrame named 'mor' found. "
                "Please pass df or create 'mor'."
            )
        df = globals()['mor']

    if not isinstance(df, pd.DataFrame):
        raise TypeError("df must be a pandas DataFrame or None.")

    cols = list(df.columns)
    if len(cols) == 0:
        raise ValueError("Provided DataFrame has no columns.")

    # The FIRST column is treated as the ID column and excluded from trait selection
    id_col = cols[0]
    data_cols = cols[1:]  # candidate trait columns

    # select traits
    if trait_cols is not None:
        if not isinstance(trait_cols, (list, tuple)):
            raise TypeError("trait_cols must be a list/tuple of column names or None.")
        missing = [c for c in trait_cols if c not in cols]
        if missing:
            raise KeyError(f"The following requested columns do not exist in the DataFrame: {missing}")
        # disallow using the id column as a trait
        if id_col in trait_cols:
            raise ValueError(f"The first column ('{id_col}') is treated as ID and cannot be used as a trait.")
        non_numeric = [c for c in trait_cols if not pd.api.types.is_numeric_dtype(df[c])]
        if non_numeric:
            raise TypeError(f"The following columns are not numeric: {non_numeric}")
        selected_traits = list(trait_cols)
    else:
        # auto-select numeric columns excluding the first column
        selected_traits = [c for c in data_cols if pd.api.types.is_numeric_dtype(df[c])]

    if not selected_traits:
        numeric_cols = [c for c in data_cols if pd.api.types.is_numeric_dtype(df[c])]
        raise ValueError(
            "No selectable traits found (selected_traits is empty). "
            f"Numeric columns available (excluding first column '{id_col}'): {numeric_cols}."
        )

    # compute ranges = max - min for the selected traits
    ranges = df[selected_traits].max() - df[selected_traits].min()
    ranges = ranges[selected_traits]  # ensure same order/index

    results = []
    nrows = len(df)

    # prepare individual names (use the values from the first column; fallback to 1-based indices)
    id_values = df.iloc[:, 0].astype(str).tolist()
    use_ids = True  # we always have a first column to use as label

    for i, j in itertools.combinations(range(nrows), 2):
        if use_ids:
            ind_i = id_values[i]
            ind_j = id_values[j]
            pair_label = f"{ind_i}_{ind_j}"
        else:
            pair_label = f"{i+1}_{j+1}"

        row = {"pair": pair_label}
        for col in selected_traits:
            xi = df.iloc[i][col]
            xj = df.iloc[j][col]
            R = ranges.loc[col]

            if pd.isna(xi) or pd.isna(xj):
                sim = float("nan")
            else:
                diff = abs(xi - xj)
                if pd.isna(R) or R == 0:
                    sim = 1.0
                else:
                    sim = (R - diff) / R
                    # clip to [0,1]
                    if sim < 0:
                        sim = 0.0
                    elif sim > 1:
                        sim = 1.0

            row[col] = round(sim, decimals) if pd.notna(sim) else sim

        results.append(row)

    # create and export global z
    global z, phenotypic_similarity_table
    z = pd.DataFrame(results)
    phenotypic_similarity_table = z
    return z


In [22]:
phenotypic_similarity(traits)

,pair,grain.yield,grain.number,seed.size,anthesis,silking,plant.height,tassel.height,ear.height
0,11430_A3,0.9207,0.9241,0.7255,0.9132,0.6998,0.8479,0.9627,0.8956
1,11430_A310,0.8040,0.8015,0.9671,0.9158,0.9134,0.8502,0.9113,0.5909
2,11430_A347,0.7754,0.8110,0.9293,0.7798,0.8225,0.8968,0.9655,0.8309
3,11430_A374,0.8544,0.6121,0.6433,0.7695,0.7868,0.9123,0.9908,0.8321
4,11430_A375,0.9854,0.8945,0.7859,0.9044,0.8835,0.8780,0.9141,0.8528
...,...,...,...,...,...,...,...,...,...
30130,W64A_W95115,0.8593,0.8048,0.9819,0.9739,0.9564,0.9977,0.8518,0.9363
30131,W64A_Wf9,0.9790,0.9505,0.9548,0.8894,0.8035,0.7762,0.7962,0.8598
30132,W9_W95115,0.8969,0.9048,0.6093,0.8366,0.8702,0.9675,0.9381,0.9644
30133,W9_Wf9,0.7772,0.9496,0.5460,0.6999,0.7174,0.8109,0.7098,0.8317


In [32]:
def heritability(
    r_df: Optional[pd.DataFrame] = None,
    z_df: Optional[pd.DataFrame] = None,
    r_col: str = "relatedness",
    key: str = "pair",
    F: Optional[float] = None,
    ddof: int = 0,
    n_bootstrap: int = 1000,
    ci: float = 0.95,
    random_state: Optional[int] = None
) -> pd.DataFrame:
    """
    Computes per-trait heritability with bootstrap standard errors and confidence intervals.

    Behavior:
      - If r_df is None, tries to use global variable `r`.
      - If z_df is None, tries to use global variable `z`.
      - If F is None, tries to use global variable `F`.
      - Raises ValueError with clear message if any required input is missing.

    Returns a DataFrame with columns:
      - trait
      - heritability (point estimate)
      - standard_error (bootstrap SE)
      - ci_lower (bootstrap percentile CI)
      - ci_upper
    Also assigns the returned DataFrame to global `heritability_table`.
    """

    # --- fallback to globals if not provided ---
    if r_df is None:
        if 'r' in globals():
            r_df = globals()['r']
        else:
            raise ValueError("r_df not provided and no global 'r' found. Please pass r_df or create global 'r'.")

    if z_df is None:
        if 'z' in globals():
            z_df = globals()['z']
        else:
            raise ValueError("z_df not provided and no global 'z' found. Please pass z_df or create global 'z'.")

    if F is None:
        if 'F' in globals():
            F = globals()['F']
        else:
            raise ValueError("F not provided and no global 'F' found. Please pass F or run heterozygosity() first.")
    F = float(F)

    # ------------ Merge ----------------------
    merged = pd.merge(r_df[[key, r_col]], z_df, on=key, how="inner")
    if merged.empty:
        raise ValueError("Merge resulted in empty dataframe. Check keys in r_df and z_df.")

    n_pairs = merged.shape[0]

    # ------------ Extract and validate r ------
    r = pd.to_numeric(merged[r_col], errors="coerce")
    if r.isna().any():
        raise ValueError(f"Non-numeric values found in relatedness column '{r_col}'.")

    var_r = r.var(ddof=ddof)
    if var_r == 0 or np.isclose(var_r, 0):
        raise ValueError("Variance of relatedness is zero; cannot compute heritability.")

    # ------------ Trait columns ---------------
    trait_cols = [c for c in merged.columns if c not in {key, r_col}]
    if not trait_cols:
        raise ValueError("No trait columns found after merging r_df and z_df.")

    # ------------ Center data -----------------
    r_centered = r - r.mean()
    z = merged[trait_cols].apply(pd.to_numeric, errors="coerce")
    if z.isna().any().any():
        raise ValueError("Non-numeric values detected in trait columns after conversion.")
    z_centered = z - z.mean()

    # ------------ Point estimates --------------
    covariances = (r_centered.values.reshape(-1, 1) * z_centered.values).mean(axis=0)
    h2_unadjusted = covariances / (2.0 * var_r)
    h2_point = h2_unadjusted * (1.0 + F)

    # ------------ Bootstrap --------------------
    rng = np.random.default_rng(random_state)
    B = int(n_bootstrap)
    boot_estimates = np.full((B, len(trait_cols)), np.nan)

    for b in range(B):
        idx = rng.integers(0, n_pairs, size=n_pairs)
        r_s = r.values[idx]
        z_s = z.values[idx, :]

        var_r_s = np.var(r_s, ddof=ddof)
        if var_r_s == 0 or np.isclose(var_r_s, 0):
            # skip this bootstrap replicate (invalid)
            continue

        r_s_centered = r_s - r_s.mean()
        z_s_centered = z_s - z_s.mean(axis=0)

        cov_s = (r_s_centered.reshape(-1, 1) * z_s_centered).mean(axis=0)
        h2_unadj_s = cov_s / (2.0 * var_r_s)
        h2_s = h2_unadj_s * (1.0 + F)

        boot_estimates[b, :] = h2_s

    # Remove invalid bootstrap iterations
    valid = ~np.all(np.isnan(boot_estimates), axis=1)
    boot_valid = boot_estimates[valid]

    if boot_valid.shape[0] == 0:
        raise RuntimeError("All bootstrap replicates invalid (variance of r was zero in all replicates).")

    # ------------ Standard errors + CI ---------
    se = np.nanstd(boot_valid, axis=0, ddof=1)

    alpha = (1 - ci) / 2
    ci_lower = np.nanpercentile(boot_valid, 100 * alpha, axis=0)
    ci_upper = np.nanpercentile(boot_valid, 100 * (1 - alpha), axis=0)

    # ------------ Final output -----------------
    out = pd.DataFrame({
        "trait": trait_cols,
        "heritability": h2_point,
        "standard_error": se,
        "ci_lower": ci_lower,
        "ci_upper": ci_upper
    })
    
    out = out.round(4)
    
    # assign to global convenience variable
    globals()['heritability_table'] = out[["trait", "heritability", "standard_error", "ci_lower", "ci_upper"]]
    print('n_bootstrap:', n_bootstrap)
    print('')
    print('heritability_table:')
    print('')
    print(heritability_table)
    print('')
    print('To access more results, type: F, heterozygosity_table, relatedness_table, phenotypic_similarity_table, heritability_table.')
    print('To save all data, type:  save_run()')
    return globals()['heritability_table']


In [33]:
def h2(gen, mor):
    snp_heterozygosity(gen)
    snp_pairwise_relatedness(gen)
    phenotypic_similarity(mor)
    heritability()

In [34]:
h2(markers,traits)

Inbreeding coefficient F: 0.9740991551394081

Step 1/6: Checking input dataframe...
Step 2/6: Extracting individual IDs (first column)...
Step 3/6: Converting SNP data to numeric format...
Step 4/6: Computing correlation matrix (this may take a while)...
Step 5/6: Cleaning correlation matrix (replacing NaN with 0.0)...
Step 6/6: Building pairwise relatedness output...
Done! Pairwise relatedness successfully computed.
n_bootstrap: 1000

heritability_table:

           trait  heritability  standard_error  ci_lower  ci_upper
0    grain.yield        0.2164          0.0082    0.2009    0.2331
1   grain.number        0.1716          0.0079    0.1571    0.1880
2      seed.size        0.1145          0.0067    0.1013    0.1273
3       anthesis        0.2049          0.0088    0.1888    0.2226
4        silking        0.2150          0.0082    0.1990    0.2304
5   plant.height        0.0469          0.0078    0.0319    0.0619
6  tassel.height        0.0679          0.0091    0.0502    0.0855
7  

In [35]:
F

0.9740991551394081

In [38]:
heritability_table

,trait,heritability,standard_error,ci_lower,ci_upper
0,grain.yield,0.2164,0.0082,0.2009,0.2331
1,grain.number,0.1716,0.0079,0.1571,0.1880
2,seed.size,0.1145,0.0067,0.1013,0.1273
3,anthesis,0.2049,0.0088,0.1888,0.2226
4,silking,0.2150,0.0082,0.1990,0.2304
5,plant.height,0.0469,0.0078,0.0319,0.0619
6,tassel.height,0.0679,0.0091,0.0502,0.0855
7,ear.height,0.0535,0.0085,0.0363,0.0695


In [43]:
def detect_markers_type(df):
    global markers_type  # create/update global variable
    
    marker_cols = df.columns[1:]

    # Detect if any marker contains "/"
    contains_slash = any(
        "/" in str(v)
        for v in df[marker_cols].stack()
    )

    # Assign marker type
    if contains_slash:
        markers_type = "microsatellites"
        print("Type of markers detected: Microsatellites")
    else:
        markers_type = "SNPs"
        print("Type of markers detected: SNPs")

In [44]:
detect_markers_type(markers)

Type of markers detected: SNPs


In [45]:
markers_type

'SNPs'